# Chapter 6 Introducing the ML Package

In the previous chapter, we worked with the MLlib package in Spark that operated strictly on RDDs. In this chapter, we move to the ML part of Spark that operates strictly on DataFrames. Also, according to the Spark documentation, the primary machine learning API for Spark is now the DataFrame-based set of models contained in the `spark.ml` package.

In this chapter, you will learn how to do the following:
- Prepare transformers, estimators, and pipelines
- Predict the chances of infant survival using models available in the ML package
- Evaluate the performance of the model
- Perform parameter hyper-tuning
- Use other machine-learning models available in the package

## Overview of the package

At the top level, the package exposes three main abstract classes: a `Transformer`, an `Estimator`, and a `Pipeline`. 

### Transformer

The `Transformer` class, like the name suggests, transforms your data by (normally) appending a new column to your DataFrame.

At the high level, when deriving from the `Transformer` abstract class, each and every new Transformer needs to implement a .transform(...) method. The method, as a first and normally the only obligatory parameter, requires passing a DataFrame to be transformed. This, of course, varies method-by-method in the ML package: other popular parameters are `inputCol` and `outputCol`; these, however, frequently default to some predefined values, such as, for example, 'features' for the `inputCol` parameter.

There are many `Transformers` offered in the `spark.ml.feature` and we will briefly describe them here (before we use some of them later in this chapter):

- `Binarizer`: Given a threshold, the method takes a continuous variable and transforms it into a binary one.
- `Bucketizer`: Similar to the `Binarizer` , this method takes a list of thresholds (the `splits` parameter) and transforms a continuous variable into a multinomial one.
- `ChiSqSelector`: For the categorical target variables (classification models), this feature allows you to select a predefined number of features (parameterized by the `numTopFeatures` parameter) that explain the variance in the target the best. The selection is done, as the name of the method suggests, using a Chi-Square test. It is one of the two-step methods: first, you need to `.fit()` your data (so the method can calculate the Chi-square tests). Calling the `.fit()` method (you pass your DataFrame as a parameter) returns a `ChiSqSelectorModel` object that you can then use to transform your DataFrame using the `.tranform()` method.
- `CountVectorizer`: This is useful for a tokenized text(such as `[['Learning','PySpark','with','us'],['us','us', 'us']`) It is one of two-step methods: first, you need to `.fit(.)`, that is, learnthe patterns from your dataset, before you can `.transform(.)` with the `CountVectorizerModel` returned by the `.fit(.)` method. The output from this transformer, for the tokenized text presented previously would look similar to this:`[(4,[0,1,2,3][1.0,1.0,1.0,1.0]),(4,[3],[3.0])]`
- `DCT`: The Discrete Cosine transform takes a vector of real values and returns a vector of the same length, but with the sum of cosine functions oscillating at different frequencies Such transformations are useful to extract some underlying frequencies in your data or in data compression
- `ElementwiseProduct`: a method that returns a vector with elements that are products of the vector passed to the method and a vector passed as the `scalingVec` parameter For example, if you had a [10.0,3.0,15.0] vector and your `scalingVec` was `[0.99, 3.30, 0.66]`, then the vector you would get would look as follows: `[9.9, 9.9, 9.9]`
- `HashingTF`: A hashing trick transformer that takes a list of tokenized text and returns a vector(of predefined length)with counts. From PySpark's documentation: Since a simple modulo is used to transform the hash function to a column index, it is advisable to use a power of two as the `numFeatures` parameter; otherwise the features will not be mapped evenly to the columns. "
- `IDF`: This method computes an Inverse Document Frequency for a list of documents. note that the documents need to already be represented as a vector(for example, using either the `HashingTF` or `CountVectorizer`)
- `Index Tostring`: A complement to the `stringIndexer` method It uses the encoding from the `stringIndexerModel` object to reverse the string index to original values. As an aside please note that this sometimes does not work and you need to specify the values from the stringIndexer
- `MaxAbsScaler`: Rescales the data to be within the `[-1.0, 1.0]` range thus it does not shift the center of the data)
- `MinMaxscaler`: This is similar to the maxabsscaler with the difference that it scales the data to be in the `[0.0, 1.0]` range
- `NGram`: This method takes a list of tokenized text and returns n-grams: pairs triples, or n-mores of subsequent words. For example, if you had a `['good','morning','Robin','Williams']` vector you would get the following output:`['good morning', 'morning Robin', 'Robin williams']`
- `Normalizer`: This method scales the data to be of unit norm using the p-norm value(by default, it is L2)
- `OneHotEncoder`: This method encodes a categorical column to a column of binary vectors
- `PCA`: Performs the data reduction using principal component analysis
- `PolynomialExpansion`: Performs a polynomial expansion of a vector. For example, if you had a vector symbolically written as `[x, y, zl` , the method would produce the following expansion: `[x, x*x,y, x*y, y*y,z,x*z, z*z]`
- `QuantileDiscretizer`: Similar to the `Bucketizer` method but instead of passing the `splits` parameter, you pass the `numBuckets` one. The method then decides, by calculating approximate quantiles over your data, what the `splits`
should be.
- `RegexTokenizer`: This is a string tokenizer using regular expressions
- `RFormula`: For those of you who are avid r users, you can pass a formula such as `vec ~ alpha*3+beta` (assuming your DataFrame has the alpha and beta columns and it will produce the vec column given the expression
- `SQLTransformer`: Similar to the previous but instead of r-like formulas you can use SQl syntax
    > The `FROM` statement should be selecting from `__THIS__`, indicating you are accessing the DataFrame. For example: `SELECT alpha * 3 + beta AS vec FROM __THIS__`.
- `StandardScaler`: Standardizes the column to have a 0-mean and standard deviation equal to 1
- `StopWordsRemover`: Removes stop words(such as 'the', 'a)from a tokenized text
- `StringIndexer`: Given a list of all the words in a column, this will produce a vector of indices
- `Tokenizer`: This is the default tokenizer that converts the string to lower case and then splits on space(s)
- `VectorAssembler`: This is a highly useful transformer that collates multiple numeric(vectors included) columns into a single column with a vector representation. For example, if you had three columns in your Data frame

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Chapter6") \
    .getOrCreate()
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=Chapter6>

In [3]:
import pyspark.ml.feature as ft

df = spark.createDataFrame([(12, 10, 3), (1, 4, 2)],['a', 'b', 'c'])

ft.VectorAssembler(inputCols=['a', 'b', 'c'],outputCol='features').transform(df).select('features').collect()

[Row(features=DenseVector([12.0, 10.0, 3.0])),
 Row(features=DenseVector([1.0, 4.0, 2.0]))]

- `VectorIndexer`: This is a method for indexing categorical columns into a vector of indices. It works in a column-by-column fashion, selecting distinct values from the column, sorting and returning an index of the value from the map instead of the original value
- `VectorSlicer`: Works on a feature vector either dense or sparse: given a list of indices, it extracts the values from the feature vector
- `Word2Vec`: This method takes a sentence(string)as an input and transforms it into a map of (string, vector) format, a representation that is useful in natural language processing

> Note that there are many methods in the ML package that have an E letter next to it; this means the method is currently in beta (or Experimental) and it sometimes might fail or produce erroneous results. Beware.

### Estimators

`Estimators` can be thought of as statistical models that need to be estimated to make predictions or classify your observations.

If deriving from the abstract `Estimator` class, the new model has to implement the `.fit(.)` method that fits the model given the data found in a `DataFrame` and some default or user-specified parameters.

#### Classification 

The ML package provides a data scientist with seven classification models to choose from. These range from the simplest ones (such as logistic regression) to more sophisticated ones. We will provide short descriptions of each of them in the following section:

- `LogisticRegression`: The benchmark model for classification. The logist regression uses a logit function to calculate the probability of an observation belonging to a particular class. At the time of writing the PySpark ML supports only binary classification problems.
- `DecisionTreeclassifier`: a classifier that builds a decision tree to predict a class for an observation. Specifying the `maxDepth` parameter limits the depth the tree grows, the minInstancePerNode determines the minimum number of observations in the tree node required to further split, the `maxBins` parameter specifies the maximum number of bins the continuous variables will be split into, and the impurity specifies the metric to measure and calculate the information gain from the split
- `GBrClassifier`: a `Gradient Boosted Trees` model for classification. The model belongs to the family of ensemble models: models that combine multiple weak predictive models to form a strong one. At the moment, the `GBTClassifier` model supports binary labels, and continuous and categorical features.
- `RandomForestclassifier`: This model produces multiple decision trees (hence the name-forest)and uses the mode output of those decision trees to classify observations. The `RandomForestclassifier` supports both binary and multinomial labels
- `NaiveBayes`: Based on the bayes theorem this model uses conditional probability theory to classify observations. The Naive Bayes model in PySpark ML supports both binary and multinomial labels
- `MultilayerPerceptronclassifier`: a classifier that mimics the nature of a human brain. Deeply rooted in the Artificial Neural Networks theory, the model is a black-box, that is, it is not easy to interpret the internal parameters of the model. The model consists, at a minimum, of three, fully connected layers(a parameter that needs to be specified when creating the model object)of artificial neurons: the input layer(that needs to be equal to the number of features in your dataset), a number of hidden layers(at least one), and an output layer with the number of neurons equal to the number of categories in your label. All the neurons in the input and hidden layers have a `sigmoid` activation function, whereas the activation function of the neurons in the output layer is `softmax`
- `OneVsRest`: A reduction of a multiclass classification to a binary one. For example, in the case of a multinomial label, the model can train multiple binary logistic regression models. For example, if `label` = 2, the model will build a logistic regression where it will convert the `label == 2` to 1(all remaining label values would be set to 0) and then train a binary model. All the models are then scored and the model with the highest probability wins.

#### Regression

There are seven models available for regression tasks in the PySpark ML package. As with classification, these range from some basic ones(such as the obligatory linear regression) to more complex ones

- `AFTSurvivalRegression`: Fits an Accelerated Failure Time regression model. It is a parametric model that assumes that a marginal effect of one of the features accelerates or decelerates a life expectancy(or process failure) It is highly applicable for the processes with well-defined stages
- `DecisionTreeRegressor`: Similar to the model for classification with an obvious distinction that the label is continuous instead of binary (or multinomial)
- `GBTRegressor`: As with the `DecisionTreeRegressor`, the difference is the data type of the label
- `GeneralizedLinearRegression`: A family of linear models with differing kernel functions(link functions). In contrast to the linear regression that assumes normality of error terms the glm allows the label to have different error term distributions: the GeneralizedLinearRegression model from the PySpark ML package supports gaussian, binomial, gamma, and poisson families of error distributions with a host of different link functions
- `IsotonicRegression:` A type of regression that fits a free-form, non decreasing line to your data. It is useful to fit the datasets with ordered and increasing observations
- `LinearRegression`: The most simple of regression models, it assumes a linear relationship between features and a continuous label, and normality of error terms
- `RandomForestRegressor`: Similar to either `DecisionTreeRegressor` or `GBTRegressor`, the `RandomForestRegressor` fits a continuous label instead of a discrete one


#### Clustering

Clustering is a family of unsupervised models that are used to find underlying patterns in your data. The PySpark ML package provides the four most popular models at the moment:

- `BisectingKMeans`: A combination of the k-means clustering method and hierarchical clustering. The algorithm begins with all observations in a single cluster and iteratively splits the data into k clusters
- `KMeans`: This is the famous k-mean algorithm that separates data into k clusters, iteratively searching for centroids that minimize the sum of square distances between each observation and the centroid of the cluster it belongs to
- `GaussianMixture`: This method uses k Gaussian distributions with unknown parameters to dissect the dataset Using the `Expectation-Maximization` algorithm, the parameters for the Gaussians are found by maximizing the log-likelihood function
    > Beware that for datasets with many features this model might perform poorly due to the curse of dimensionality and numerical issues with Gaussian distributions.
- `LDA`: This model is used for topic modeling in natural language processing applications.

There is also one recommendation model available in PySpark ML, but we will refrain from describing it here.

### Pipeline

A `Pipeline` in PySpark ML is a concept of an end-to-end transformation-estimation process (with distinct stages) that ingests some raw data (in a DataFrame form), performs the necessary data carpentry (transformations), and finally estimates a statistical model (estimator).

> A Pipeline can be purely transformative, that is, consisting of Transformers only.

A `Pipeline` can be thought of as a chain of multiple discrete stages. When a `.fit(...)` method is executed on a `Pipeline` object, all the stages are executed in the order they were specified in the stages parameter; the stages parameter is a list of `Transformer` and `Estimator` objects. The `.fit(...)` method of the `Pipeline` object executes the `.transform(...)` method for the `Transformers` and the `.fit(...)` method for the Estimators.

Normally, the output of a preceding stage becomes the input for the following stage: when deriving from either the `Transformer` or `Estimator` abstract classes, one needs to implement the `.getOutputCol()` method that returns the value of the `outputCol` parameter specified when creating an object.

## Predicting the chances of infant survival with ML

### Load Data



In [6]:
import pyspark.sql.types as typ

labels = [
    ('INFANT_ALIVE_AT_REPORT', typ.IntegerType()),
    ('BIRTH_PLACE', typ.StringType()),
    ('MOTHER_AGE_YEARS', typ.IntegerType()),
    ('FATHER_COMBINED_AGE', typ.IntegerType()),
    ('CIG_BEFORE', typ.IntegerType()),
    ('CIG_1_TRI', typ.IntegerType()),
    ('CIG_2_TRI', typ.IntegerType()),
    ('CIG_3_TRI', typ.IntegerType()),
    ('MOTHER_HEIGHT_IN', typ.IntegerType()),
    ('MOTHER_PRE_WEIGHT', typ.IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', typ.IntegerType()),
    ('MOTHER_WEIGHT_GAIN', typ.IntegerType()),
    ('DIABETES_PRE', typ.IntegerType()),
    ('DIABETES_GEST', typ.IntegerType()),
    ('HYP_TENS_PRE', typ.IntegerType()),
    ('HYP_TENS_GEST', typ.IntegerType()),
    ('PREV_BIRTH_PRETERM', typ.IntegerType())
]

schema = typ.StructType([
    typ.StructField(e[0], e[1], False) for e in labels
])

births = spark.read.csv('data/births_transformed.csv.gz', 
                        header=True, 
                        schema=schema)

### Creating Transformers 

Since statistical models can only operate on numeric data, we will have to encode the `BIRTH_PLACE` variable.

To encode the `BIRTH_PLACE` column, we will use the `OneHotEncoder` method. However, the method cannot accept `StringType` columns; it can only deal with numeric types so first we will cast the column to an `IntegerType`:

In [7]:
births = births \
    .withColumn(       'BIRTH_PLACE_INT', 
                births['BIRTH_PLACE'] \
                    .cast(typ.IntegerType()))

Having done this, we can now create our first Transformer:

In [8]:
encoder = ft.OneHotEncoder(
    inputCol='BIRTH_PLACE_INT', 
    outputCol='BIRTH_PLACE_VEC')

Let's now create a single column with all the features collated together. We will use the `VectorAssembler` method:

In [9]:
featuresCreator = ft.VectorAssembler(
    inputCols=[
        col[0] 
        for col 
        in labels[2:]] + \
    [encoder.getOutputCol()], 
    outputCol='features'
)

The `inputCols` parameter passed to the `VectorAssembler` object is a list of all the columns to be combined together to form the `outputCol` —the 'features'. Note that we use the output of the encoder object (by calling the `.getOutputCol()` method), so we do not have to remember to change this parameter's value should we change the name of the output column in the encoder object at any point.


### Creating an estimator

In this example, we will (once again) use the logistic regression model.

In [10]:
import pyspark.ml.classification as cl

logistic = cl.LogisticRegression(
    maxIter=10, 
    regParam=0.01, 
    labelCol='INFANT_ALIVE_AT_REPORT')

We would not have to specify the `labelCol` parameter if our target column had the name 'label'. Also, if the output of our `featuresCreator` was not called 'features', we would have to specify the `featuresCo`l by (most conveniently) calling the `getOutputCol()` method on the `featuresCreator` object.

### Creating a pipeline
